In [3]:
import pandas as pd
import glob
import pickle
from io import StringIO
import re

DATA_BASE_PATH = "./hein-daily/"
DATA_SAVE_PATH = "./preprocessed_data/"

In [4]:
def write_to_disk(data, full_path):
    with open(full_path, 'wb') as file:
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

def read_from_disk(full_path):
    with open(full_path, 'rb') as file:
        data = pickle.load(file)
        return data

In [78]:
def read_raw_files(base_path, filename_description = "", filename_rule = "", splitby_rule = "", take_splitelem = 0, delimiter = "|", encoding = None, replace_first_n_delims = 0):
    
    all_files = None
    
    for path in glob.glob(base_path + filename_rule):
        print(path)
        
        file_number = path.split("/")[-1].split(splitby_rule)[take_splitelem].split(".txt")[0]
        
        # in case of structurally corrupted file 
        if replace_first_n_delims > 0:
            custom_delim = "°@@°"
            dummy_file = replace_delimiters(path, encoding = encoding, replace_first_n = replace_first_n_delims, find_delim = delimiter, replace_with = custom_delim) 
            df = pd.read_csv(dummy_file, delimiter = custom_delim, encoding = encoding)
    
        # healthy file 
        else:
            df = pd.read_csv(path, delimiter = delimiter, encoding = encoding)
       
    
        # add file info 
        df[filename_description] = file_number
        
        if all_files is None:
            all_files = df.copy(deep = True)
        else:
            all_files = pd.concat([all_files,df])
            
        
    return all_files.reset_index(drop=True)




def replace_delimiters(file_path, encoding = None, replace_first_n = 1, find_delim = "|", replace_with = "°@@°"):
    '''
    In some files, the delimiter occurs again within the content, corrupting the file structure.
    Thus, replace the first n correctly placed delimiters with another delimiter. 
    Credit: https://stackoverflow.com/a/48672386 
    '''
    
    file_dummy = StringIO()
    with open(file_path, encoding = encoding) as file:
        for line in file:
           
            new_line = line.replace(find_delim, replace_with,1)
            print(new_line, file=file_dummy)
    
    file_dummy.seek(0)
    
    return file_dummy


# Read

In [84]:
#speakermaps = read_raw_files(base_path = DATA_BASE_PATH, filename_description = "speakermap_filename", filename_rule = "*SpeakerMap.*", splitby_rule = "_SpeakerMap", take_splitelem = 0, delimiter = "|")   
#byparty_2grams = read_raw_files(base_path = DATA_BASE_PATH, filename_description = "byparty_2gram_filename", filename_rule = "*byparty_2gram_*", splitby_rule = "byparty_2gram_", take_splitelem = 1, delimiter = "|")
#byspeaker_2grams = read_raw_files(base_path = DATA_BASE_PATH, filename_description = "byspeaker_2gram_filename", filename_rule = "*byspeaker_2gram_*", splitby_rule = "byspeaker_2gram_", take_splitelem = 1, delimiter = "|")
#descriptions = read_raw_files(base_path = DATA_BASE_PATH, filename_description = "descriptions_filename", filename_rule = "*descr_*", splitby_rule = "descr_", take_splitelem = 1, delimiter = "|")   
#speeches = read_raw_files(base_path = DATA_BASE_PATH, filename_description = "speeches_filename", filename_rule = "*speeches_*", splitby_rule = "speeches_", take_splitelem = 1, delimiter = "|", encoding = "iso-8859-1", replace_first_n_delims = 1)   

# Write

In [81]:
#write_to_disk(speakermaps, DATA_SAVE_PATH + "raw_speakermaps.pickle")
#write_to_disk(byparty_2grams, DATA_SAVE_PATH + "raw_byparty_2grams.pickle")
#write_to_disk(byspeaker_2grams, DATA_SAVE_PATH + "raw_byspeaker_2grams.pickle")
#write_to_disk(descriptions, DATA_SAVE_PATH + "raw_descriptions.pickle")
#write_to_disk(speeches, DATA_SAVE_PATH + "raw_speeches.pickle")